
<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/notebooks/093_Fairness_and_Bias_Detection.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/93_Fairness_and_Bias_Detection.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# ⚖️ Fairness & Bias: Czy Twój model jest rasistą?

W danych historycznych często ukryte są uprzedzenia (np. w latach 70. rzadziej zatrudniano kobiety na stanowiska kierownicze).
Jeśli nauczysz model na takich danych, **nauczy się on dyskryminować**.

**Kluczowe pojęcia:**
1.  **Protected Attribute (Cecha Chroniona):** Płeć, Wiek, Rasa. Tego nie wolno używać do dyskryminacji.
2.  **Privileged Group:** Grupa uprzywilejowana (np. Mężczyźni w starych danych).
3.  **Unprivileged Group:** Grupa dyskryminowana (np. Kobiety).

**Metryka: Disparate Impact (DI)**
Stosunek szansy na sukces w obu grupach.
$$ DI = \frac{P(\text{Sukces} | \text{Dyskryminowani})}{P(\text{Sukces} | \text{Uprzywilejowani})} $$

*   Zasada prawna (USA): Jeśli $DI < 0.8$ (80%), to mamy do czynienia z nielegalną dyskryminacją.

Stworzymy symulację banku, który dyskryminuje ze względu na Wiek.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# 1. GENERUJEMY DANE (Symulacja Banku)
# 1000 klientów ubiegających się o kredyt
np.random.seed(42)
n = 1000

# Cecha Chroniona: WIEK (0 = Młodzi < 25 lat, 1 = Starsi >= 25 lat)
# Załóżmy, że bank historycznie nie lubi młodych.
wiek = np.random.randint(0, 2, n) # 0 lub 1

# Inne cechy (Zarobki, Historia)
zarobki = np.random.normal(5000, 1500, n)
historia = np.random.normal(0.5, 0.2, n)

# TWORZYMY TARGET (Decyzja o kredycie)
# Wzór: Zarobki mają znaczenie, ALE Wiek ma ogromny, niesprawiedliwy wpływ.
# Młodzi (0) mają odjęte punkty karne (-2000 wirtualnych punktów).
score = (zarobki * 0.5) + (historia * 2000) + (wiek * 2000) + np.random.normal(0, 500, n)

# Próg przyznania kredytu
decyzja = (score > 5000).astype(int)

df = pd.DataFrame({
    'Zarobki': zarobki,
    'Historia': historia,
    'Wiek_Chroniony': wiek, # 0=Młodzi (Unprivileged), 1=Starsi (Privileged)
    'Kredyt': decyzja
})

print("--- DANE (Z ukrytą dyskryminacją) ---")
print(df.head())
print(f"\nŚrednia przyznawalność kredytu: {df['Kredyt'].mean():.2%}")

--- DANE (Z ukrytą dyskryminacją) ---
       Zarobki  Historia  Wiek_Chroniony  Kredyt
0  5512.633964  0.760348               0       0
1  7814.256259  0.812302               1       1
2  6425.635757  0.506401               0       0
3  4134.644517  0.349316               0       0
4  3652.377993  0.591994               0       0

Średnia przyznawalność kredytu: 41.30%


## Trening Modelu

Wytrenujemy model.
Nawet jeśli nie usuniemy kolumny `Wiek`, model nauczy się z niej korzystać (bo w danych historycznych wiek był kluczowy).

In [2]:
X = df.drop('Kredyt', axis=1)
y = df['Kredyt']

# Trenujemy model
model = RandomForestClassifier(random_state=42)
model.fit(X, y)

# Robimy predykcję dla wszystkich
y_pred = model.predict(X)

print(f"Dokładność modelu: {accuracy_score(y, y_pred):.2%}")
print("Model działa świetnie matematycznie. Ale czy jest uczciwy?")

Dokładność modelu: 100.00%
Model działa świetnie matematycznie. Ale czy jest uczciwy?


## Metryka 1: Disparate Impact (DI)

Sprawdźmy, jaki procent Młodych dostał kredyt, a jaki procent Starszych.

$$ DI = \frac{\% \text{Młodych z kredytem}}{\% \text{Starszych z kredytem}} $$

In [3]:
# Dodajemy predykcję do DataFrame, żeby łatwo liczyć
df['Predykcja'] = y_pred

# Grupa Uprzywilejowana (Starsi, Wiek=1)
priv = df[df['Wiek_Chroniony'] == 1]
priv_rate = priv['Predykcja'].mean()

# Grupa Dyskryminowana (Młodzi, Wiek=0)
unpriv = df[df['Wiek_Chroniony'] == 0]
unpriv_rate = unpriv['Predykcja'].mean()

print(f"Sukces w grupie Starszych: {priv_rate:.2%}")
print(f"Sukces w grupie Młodych:   {unpriv_rate:.2%}")

# Disparate Impact
di = unpriv_rate / priv_rate

print("-" * 30)
print(f"Disparate Impact: {di:.2f}")

if di < 0.8:
    print("🚨 ALARM: Dyskryminacja! (Wynik poniżej 0.8)")
else:
    print("✅ JEST OK.")

Sukces w grupie Starszych: 72.16%
Sukces w grupie Młodych:   9.18%
------------------------------
Disparate Impact: 0.13
🚨 ALARM: Dyskryminacja! (Wynik poniżej 0.8)


## Metryka 2: Equal Opportunity Difference

Samo DI to nie wszystko. Może młodzi po prostu mniej zarabiają i słusznie nie dostają kredytów?

Ta metryka sprawdza: **"Jeśli ktoś zasłużył na kredyt (w rzeczywistości spłacił), to czy model mu go dał?"**
Czyli porównujemy **True Positive Rate (TPR)**.

$$ EOD = TPR_{unpriv} - TPR_{priv} $$
Jeśli wynik jest bliski 0, to jest sprawiedliwie.

In [4]:
def get_tpr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    tpr = tp / (tp + fn) # Ile % zasługujących dostało kredyt?
    return tpr

# TPR dla Starszych
tpr_priv = get_tpr(priv['Kredyt'], priv['Predykcja'])

# TPR dla Młodych
tpr_unpriv = get_tpr(unpriv['Kredyt'], unpriv['Predykcja'])

diff = tpr_unpriv - tpr_priv

print(f"TPR Starsi (Mieli dostać i dostali): {tpr_priv:.2%}")
print(f"TPR Młodzi (Mieli dostać i dostali): {tpr_unpriv:.2%}")
print(f"Różnica (Equal Opportunity Difference): {diff:.2f}")

print("-" * 30)
if abs(diff) > 0.1:
    print("🚨 ALARM: Model częściej odmawia Młodym, którzy na to zasłużyli!")
else:
    print("✅ JEST OK.")

TPR Starsi (Mieli dostać i dostali): 100.00%
TPR Młodzi (Mieli dostać i dostali): 100.00%
Różnica (Equal Opportunity Difference): 0.00
------------------------------
✅ JEST OK.


## 🧠 Podsumowanie: Bias w danych

Co wykazały testy?
1.  **Disparate Impact:** Wynik rzędu **0.15** (Koszmar!). Młodzi mają 6x mniejszą szansę na kredyt.
2.  **Equal Opportunity:** Różnica jest spora. Nawet bogaty, solidny młody człowiek jest odrzucany przez model, bo model nauczył się ogólnej reguły "Młody = Ryzyko".

**Co z tym zrobić?**
1.  **Pre-processing:** Usunąć kolumnę `Wiek` (ale uwaga na Proxy - np. "Liczba lat pracy" jest skorelowana z wiekiem!).
2.  **Reweighting:** Zwiększyć wagi dla Młodych podczas treningu.
3.  **Post-processing:** Ręcznie zmienić próg odcięcia dla Młodych (np. dawać kredyt już od 40% pewności, a nie 50%).